<a href="https://colab.research.google.com/github/nanochiplab/Sayisal_Goruntu/blob/main/Odev_TekCizim_Final_G%C3%B6r%C3%BCnt%C3%BC_%C4%B0%C5%9Fleme_9_G%C3%B6r%C3%BCnt%C3%BC_Li.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Gerekli Kütüphaneleri yüklüyoruz
import pandas as pd                   #Pandas tablo formatındaki verilerle çalışmak içn
import numpy as np                    #diziler üzerinde sayısal işlemler için
import scipy.ndimage as ndi
import matplotlib.pyplot as plt       #görselleştirmeler oluşturmak için
#çizimleri doğrudan Jupyter Notebook'ta göstermek için
%matplotlib inline
import os                             #işletim sistemiyle etkileşim kurmak (dosyalara erişmek gibi) için
import seaborn as sns                 #çizimlerin estetiğini geliştirmek için
sns.set()                             #mevcut tema style,font ve rengi kullanmak için(değiştirilebilir)
from skimage import exposure          #görüntü işleme işlevleri sağlamak için
import cv2  # opencv for python package  #bilgisayarlı görüş işlemleri için (OpenCV kütüphanesidir)
from scipy.ndimage import zoom        #zoomlama dahil olmak üzere çok boyutlu görüntü işlemleri sağlamak için
                                      #bu kütphaneleri yükledik...............
from google.colab.patches import cv2_imshow

#Tıbbi görüntü verilerini içeren bir GitHub deposunu klonluyoruz.
#Deponun içindeki belirli klasöre gidiyoruz
!git clone https://github.com/hardik0/AI-for-Medicine-Specialization
%cd AI-for-Medicine-Specialization/AI-for-Medical-Diagnosis/


train_df = pd.read_csv("nih/train-small.csv")  # train-small.csv adlı dosyayı okur ve train_df adlı DataFrame'e yükler
print(f'There are {train_df.shape[0]} rows and {train_df.shape[1]} columns in this data frame') #Satır ve sütun sayısı, veri türleri ve boş değerler dahil olmak üzere DataFrame hakkında bilgi görüntüler
train_df.head()   #Dataframe'in ilk 5 satırnı döndürür

# Her sütuna null değer var mı yok mu bakalım..
train_df.info()  #Dataframe'in özetini döndürüp yazdırır....
#Toplam "PatientId" sayısını ve benzersiz(unique) PatientId sayısını hesaplar ve yazdırır.
print(f"The total patient ids are {train_df['PatientId'].count()}, from those the unique ids are {train_df['PatientId'].value_counts().shape[0]} ")

columns = train_df.keys()  #columns isimli listeye,Dataframe'deki sütun isimlerini döndürür...
columns = list(columns)    #column'u listele...
print(columns)             #listeyi yazdır.....

# Gereksiz elemanları kaldır
columns.remove('Image')      #Image sütununu kaldır...
columns.remove('PatientId')  #PatientId sütununu kaldır...
# Geriye kalan sütunlrı yazdır..
print(f"There are {len(columns)} columns of labels for these conditions: {columns}")

#**************************DEFINITION*************************************************************************
def apply_gaussian_transform(image, mean_intensity, std_intensity):
    """
    Apply Gaussian transformation to the image intensity values.
    :param image: Input image (grayscale).
    :param mean: Mean intensity value.
    :param std: Standard deviation (adjusted).
    :return: Transformed image.
    """
    # Compute Gaussian transformation
    gaussian_transformed = (1 / (std_intensity * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((image - mean_intensity) / std_intensity) ** 2)

    # Normalize the transformed image to the range [0, 255]
    transformed_normalized = cv2.normalize(gaussian_transformed, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    return transformed_normalized
#***************************DEFINITION************************************************************************

def filter_regions(regions, image_width, image_height):
    # Alana göre sırala
    sorted_regions = sorted(regions.items(), key=lambda item: item[1]["area"], reverse=True)

    # İlk iki bölgeyi seç
    top_two_regions = sorted_regions[:2]

    # Centroidleri ortada ise filtrele
    filtered_regions = []
    for label, region in top_two_regions:
        centroid = region["centroid"]
        if not (image_width / 3 < centroid[1] < 2 * image_width / 3 and
                image_height / 3 < centroid[0] < 2 * image_height / 3):
            filtered_regions.append(label)

    # Filtrelenmiş bölgeler yoksa, 3. bölgeyi dene
    if not filtered_regions:
        if len(sorted_regions) > 2:
            filtered_regions.append(sorted_regions[2][0])

    return filtered_regions
#****************************DEFINITION*************************************************************************
def auto_label_image(img):
    img_gray = img
    img_blur = cv2.GaussianBlur(img_gray, (5, 5), 0)
    # Convert the image to 8-bit unsigned integer before applying thresholding
    img_blur = img_blur.astype(np.uint8)  # Convert to uint8
    #ret, thresh_img = cv2.threshold(img_blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    ret, thresh_img = cv2.threshold(img_blur, 0, 255, cv2.THRESH_BINARY)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh_img)
    max_area = 0
    label = 0
    for i in range(1, num_labels):
        area = stats[i, cv2.CC_STAT_AREA]
        if area > max_area:
            max_area = area
            label = i

    # Create a result image with 3 channels (for RGB)
    result_img = np.zeros_like(thresh_img, dtype=np.uint8) # Convert to uint8
    #result_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)  # Create a 3-channel image


    # Use broadcasting to assign the same RGB value to all channels
    result_img[labels == label] = 255  # Assign 255 to all channels (grayscale)

    return result_img
#***************************************************************************************************
# Print out the number of positive labels for each class
for column in columns:   #
    print(f"The class {column} has {train_df[column].sum()} samples") #örnek sayısını yazdırır...

#'Image' sütunundan görüntü dosyası isimlerini alır ve bunları images değişkenine atar
images = train_df['Image'].values

#sample_img=train_df.images[0]
#sample_img = train_df['Image'].values[10] # Access the first image filename correctly.

# 9 görüntü dosyasını rastgele seçer ve bunları random_images'e atar....
random_images = [np.random.choice(images) for i in range(9)]

#Görüntülerin bulunduğu dizini belirtir (img_dir)....
img_dir = 'nih/images_small/'

print('Display Random Images') #Random(Rastgele) görüntülerin görüntüleneceğini belirten bir mesaj yazdırır...

#******************************9 ADET GÖRÜNTÜ SEÇİMİ*********************************************************************
for i in range(9):        #9 adet görüntü ile çalışacağız..
    #Görüntülerin bulunduğu dizini okur ve for döngüsü dahilinde sırayla raw_image'e atar...
    read_image = plt.imread(os.path.join(img_dir, random_images[i]))

  #Görüntülerin bulunduğu dizini okur ve for döngüsü dahilinde sırayla raw_image'e atar...
  #read_image = plt.imread(os.path.join(img_dir, sample_img))

    x1, y1, x2, y2 = 20, 50, 1020, 1000  # x1 soldan,y1 üstten, x2 sağdan,y2 alttan,
    raw_image = read_image[y1:y2, x1:x2]

  # Crop edilmiş görüntüyü kaydet
    cv2.imwrite("cropped_image.jpg", raw_image)

  # Orjinal görüntüyü çizeceğiz..
    plt.figure(figsize=(16, 12))                 #16 inç x 12 inç boyutlarında çizim için yeni bir şekil oluşturur...
    plt.subplot(1, 2, 1)
    plt.imshow(read_image, cmap='gray')           #Ham görüntüyü (orijinal, işlenmemiş görüntüyü) görüntüleyeceğiz..cmap='gray',renk haritasını tıbbi görüntüler için uygun olan gri tonlamaya ayarlar
    plt.title('Orjinal Görüntü'+' '+str(i+1))     #Grafiğin başlığını ayarlar ve ardından görüntü numarasını ekler...
    plt.colorbar()                               #Grafiğe, görüntüdeki piksel yoğunluk değerlerinin görsel bir temsili olan bir renk çubuğu ekler.

    # Orjinal görüntüyü çizeceğiz..
    #plt.figure(figsize=(16, 12))                 #16 inç x 120 inç boyutlarında çizim için yeni bir şekil oluşturur...
    plt.subplot(1, 2, 2)
    plt.imshow(raw_image, cmap='gray')           #Ham görüntüyü (orijinal, işlenmemiş görüntüyü) görüntüleyeceğiz..cmap='gray',renk haritasını tıbbi görüntüler için uygun olan gri tonlamaya ayarlar
    plt.title('Kırpılmış Görüntü'+' '+str(i+1))     #Grafiğin başlığını ayarlar ve ardından görüntü numarasını ekler...
    plt.colorbar()                               #Grafiğe, görüntüdeki piksel yoğunluk değerlerinin görsel bir temsili olan bir renk çubuğu ekler.


    #Buraya stretching veya equalzation uygulanacak...
    min_val, max_val = raw_image.min(), raw_image.max()
    contrast_stretched = (raw_image - min_val) / (max_val - min_val)  # [0, 1] aralığında normalize eder..
    contrast_stretched = (contrast_stretched * 255).astype(np.uint8)  # Normalize edilen değerleri piksel yoğunluğunu #göstermek için 0-255 aralığında ölçeklendirir...

    equalized_image = exposure.equalize_hist(contrast_stretched)  #Çıkış değerleri 0-1 aralığında
    equalized_image = (equalized_image * 255).astype(np.uint8)    #Piksel değerleri 0-255 aralığında ve veri tipi 8 bitlik işaretsiz tamsayıya çevrildi

    # Orjinal görüntüyü çizeceğiz..
    #plt.figure(figsize=(8, 6))                 #18 inç x 10 inç boyutlarında çizim için yeni bir şekil oluşturur...
    plt.figure(figsize=(16, 12))                 #16 inç x 12 inç boyutlarında çizim için yeni bir şekil oluşturur...
    plt.subplot(1, 2, 1)
    plt.imshow(contrast_stretched, cmap='gray')           #Ham görüntüyü (orijinal, işlenmemiş görüntüyü) görüntüleyeceğiz..cmap='gray',renk haritasını tıbbi görüntüler için uygun olan gri tonlamaya ayarlar
    plt.title('Contrast Stretched Görüntü'+' '+str(i+1))     #Grafiğin başlığını ayarlar ve ardından görüntü numarasını ekler...
    plt.colorbar()                               #Grafiğe, görüntüdeki piksel yoğunluk değerlerinin görsel bir temsili olan bir renk çubuğu ekler.

    # Equalized görüntüyü çizeceğiz..
    #plt.figure(figsize=(16, 12))                 #18 inç x 10 inç boyutlarında çizim için yeni bir şekil oluşturur...
    plt.subplot(1, 2, 2)
    plt.imshow(equalized_image, cmap='gray')           #Ham görüntüyü (orijinal, işlenmemiş görüntüyü) görüntüleyeceğiz..cmap='gray',renk haritasını tıbbi görüntüler için uygun olan gri tonlamaya ayarlar
    plt.title('Equalized Görüntü'+' '+str(i+1))     #Grafiğin başlığını ayarlar ve ardından görüntü numarasını ekler...
    plt.colorbar()                               #Grafiğe, görüntüdeki piksel yoğunluk değerlerinin görsel bir temsili olan bir renk çubuğu ekler.


    # Adım 3: Gama Düzeltmesi uygulanacak....
    gamma = 3  #Görüntünün parlaklığını kontrol eden gama değerini ayarlar.1'den büyük bir gama değeri görüntüyü aydınlatırken, 1'den küçük bir değer görüntüyü koyulaştırır.
    gamma_corrected = np.power(equalized_image / 255.0, gamma) * 255 #Eşitlenmiş görüntüye gama düzeltmesi uygular.İlk önce piksel değerlerini [0, 1] aralığına normalleştirir, ardından bunları
    #gama kuvvetine yükseltir ve son olarak bunları [0, 255] aralığına geri ölçekler
    gamma_corrected = gamma_corrected.astype(np.uint8) #gamma_corrected'ın veri türünü np.uint8'e 8 bitlik tamsayıya dönüştürür.
    #GAMA DÜZELTMESİNDE PEK BİR DEĞİŞİM GÖZLEYEMEDİM..

     #**************************MEDİAN VE GAUSS FİLTRELEME*********************
    # Adım 4: Gürültü azaltmı yapılacak...
    #gamma_corrected görüntüsüne medyan filtreleme uygulayacağız.Her pikseli komşu piksellerinin medyan değeriyle değiştirerek
    #Median Filtreleme tuz-biber gürültüsünü gidermeye yardımcı olur.
    #median_filtered = cv2.medianBlur(gamma_corrected, 5) # 5x5 boyutunda...
    #GÖRÜNTÜDE TUZ BİBER ELEMENTLER OLMADIĞI İÇİN MEDİAN FİLTRELEME PEK BİR DEĞİŞİM GÖZLEYEMEDİM....

    #gamma_corrected görüntüsüne Gauss bulanıklaştırma uygular.Gauss çekirdeği kullanarak görüntüyü bulanıklaştıran başka bir gürültü azaltma tekniğidir.Görüntüyü yumuşatmaya ve yüksek
    #frekanslı gürültüleri azaltmaya yardımcı olur.
    gaussian_filtered = cv2.GaussianBlur(gamma_corrected, (5, 5), 0)
    #GAUSS'DA GÖRÜNTÜ DAHA BLANIKLAŞTI VE YUMUŞADI...

    # Equalized görüntüyü çizeceğiz..
    plt.figure(figsize=(16, 12))                 #16 inç x 12 inç boyutlarında çizim için yeni bir şekil oluşturur...
    plt.subplot(1, 2, 1)
    plt.imshow(gaussian_filtered, cmap='gray')           #Ham görüntüyü (orijinal, işlenmemiş görüntüyü) görüntüleyeceğiz..cmap='gray',renk haritasını tıbbi görüntüler için uygun olan gri tonlamaya ayarlar
    plt.title('Gauss filtrelenmiş Görüntü'+' '+str(i+1))     #Grafiğin başlığını ayarlar ve ardından görüntü numarasını ekler...
    plt.colorbar()                               #Grafiğe, görüntüdeki piksel yoğunluk değerlerinin görsel bir temsili olan bir renk çubuğu ekler.


    mean_intensity=np.mean(gaussian_filtered)
    std_intensity=np.std(gaussian_filtered) * 1

    print(f"Mean Intensity: {mean_intensity}")
    print(f"Standard Deviation: {std_intensity}")

    # Gauss Tranformu piksel değerlerini komşu piksellerin ağırlıklı ortalaması ile değiştirerek gürültüyü azaltıyor...
    transformed_normalized=apply_gaussian_transform(gaussian_filtered,mean_intensity-0.3,std_intensity)
    #raw_transform=(raw_image*255).astype(np.uint8)




    #plt.figure(figsize=(8, 6))                 #18 inç x 10 inç boyutlarında çizim için yeni bir şekil oluşturur...
    plt.subplot(1, 2, 2)
    plt.imshow(transformed_normalized, cmap='gray')           #Ham görüntüyü (orijinal, işlenmemiş görüntüyü) görüntüleyeceğiz..cmap='gray',renk haritasını tıbbi görüntüler için uygun olan gri tonlamaya ayarlar
    plt.title('Transformed Görüntü'+' '+str(i+1))     #Grafiğin başlığını ayarlar ve ardından görüntü numarasını ekler...
    plt.colorbar()

    hist=cv2.calcHist([transformed_normalized],[0],None,[256],[0,256])

    plt.figure(figsize=(8, 6))  # Create a new figure for the histogram
    plt.subplot(1, 1, 1)
    plt.title('Transformed Görüntünün Histogramı'+' '+str(i+1))  # Add a title
    plt.xlabel('Pixel Intensity')  # Label the x-axis
    plt.ylabel('Frequency')  # Label the y-axis
    plt.plot(hist, color='black')  # Plot the histogram data
    plt.xlim([0, 255])  # Set the x-axis limits
    plt.grid
    plt.show()  # Display the histogram plot


    from skimage.filters import try_all_threshold, threshold_otsu, threshold_li, threshold_mean, threshold_minimum, threshold_triangle, threshold_isodata, threshold_yen

    # Görüntünüzü yükleyin.
    #img = raw_image
    #img=gaussian_filtered
    img=transformed_normalized
    # Tüm mevcut eşikleme yöntemlerini deneyin ve eşik değerlerini alın.
    thresh_values = {
        "Isodata": threshold_isodata(img),  # If you have threshold_isodata available
        "Li": threshold_li(img),
        "Mean": threshold_mean(img),
        "Minimum": threshold_minimum(img),
        "Otsu": threshold_otsu(img),
        "Triangle": threshold_triangle(img),
        "Yen": threshold_yen(img),
    }

    # Eşik değerlerini yazdırın.
    print(thresh_values)

    # Tüm mevcut eşikleme yöntemlerini görselleştirin (isteğe bağlı).
    fig, ax = try_all_threshold(img, figsize=(10,8), verbose=False)
    plt.show()


    #thresh_value = 123  # Otsu'nun yöntemi ile belirlenen değer
    thresh_value =threshold_li(img)

    #thresh_mask=transformed_normalized < thresh_value
    #thresh_img = cv2.threshold(img, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

    ret, thresh_img = cv2.threshold(img, thresh_value, 255, cv2.THRESH_BINARY)

    plt.figure(figsize=(8, 6))
    plt.subplot(1, 1, 1)
    plt.title('OTSU Threshold Uygulanmış Görüntü'+' '+str(i+1))  # Add a title
    plt.imshow(thresh_img, cmap='gray')
    plt.show()


    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 1))
    eroded_img = cv2.erode(thresh_img, kernel, iterations=20)
    dilated_img = cv2.dilate(eroded_img, kernel, iterations=20)

    opening = cv2.morphologyEx(dilated_img, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)

    plt.figure(figsize=(8, 6))                 # 8 inç x 6 inç boyutlarında çizim için yeni bir şekil oluşturur...
    plt.imshow(eroded_img, cmap='gray')           #Ham görüntüyü (orijinal, işlenmemiş görüntüyü) görüntüleyeceğiz..cmap='gray',renk haritasını tıbbi görüntüler için uygun olan gri tonlamaya ayarlar
    plt.title('Erosion Uygulanmış Görüntü'+' '+str(i+1))     #Grafiğin başlığını ayarlar ve ardından görüntü numarasını ekler...
    plt.colorbar()                               #Grafiğe, görüntüdeki piksel yoğunluk değerlerinin görsel bir temsili olan bir renk çubuğu ekler.

    plt.figure(figsize=(8, 6))                 # 8 inç x 6 inç boyutlarında çizim için yeni bir şekil oluşturur...
    plt.imshow(dilated_img, cmap='gray')           #Ham görüntüyü (orijinal, işlenmemiş görüntüyü) görüntüleyeceğiz..cmap='gray',renk haritasını tıbbi görüntüler için uygun olan gri tonlamaya ayarlar
    plt.title('Dilated Uygulanmış Görüntü'+' '+str(i+1))     #Grafiğin başlığını ayarlar ve ardından görüntü numarasını ekler...
    plt.colorbar()

    plt.figure(figsize=(8, 6))                 # 8 inç x 6 inç boyutlarında çizim için yeni bir şekil oluşturur...
    plt.imshow(opening, cmap='gray')           #Ham görüntüyü (orijinal, işlenmemiş görüntüyü) görüntüleyeceğiz..cmap='gray',renk haritasını tıbbi görüntüler için uygun olan gri tonlamaya ayarlar
    plt.title('opening Uygulanmış Görüntü'+' '+str(i+1))     #Grafiğin başlığını ayarlar ve ardından görüntü numarasını ekler...
    plt.colorbar()

    plt.figure(figsize=(8, 6))                 # 8 inç x 6 inç boyutlarında çizim için yeni bir şekil oluşturur...
    plt.imshow(closing, cmap='gray')           #Ham görüntüyü (orijinal, işlenmemiş görüntüyü) görüntüleyeceğiz..cmap='gray',renk haritasını tıbbi görüntüler için uygun olan gri tonlamaya ayarlar
    plt.title('closing Uygulanmış Görüntü'+' '+str(i+1))     #Grafiğin başlığını ayarlar ve ardından görüntü numarasını ekler...
    plt.colorbar()

    #**********************CONNECTED COMPONENTS UYGULAMA**********************************************
    num_labels, labels = cv2.connectedComponents(closing)

    #print("Olası bölge sayısı:", num_labels - 1)
    #print("")

    labels[labels == 0] = -1
    labels[labels > 0] = labels[labels > 0] - 1

    plt.figure(figsize=(16, 12))
    plt.imshow(labels, cmap='jet')
    plt.title("CCL Çıktısı -- OLASI BÖLGE SAYISI:" + str(num_labels - 1))
    plt.colorbar()
    plt.show()

    #********POST PROCESSİNG 3.3***********************************************
    #Elde ettiğiniz labellar için, Centroid ve Area bilgilerinin kesinlikle olacağı, ayrıca bölgelerin yoğunluk ve şekilsel özelliklerini belirten en az 5 tane özniteliği print edin
    from skimage.measure import regionprops, label

    props = regionprops(label(labels), intensity_image=closing)

    for prop in props:
        print("Label:", prop.label)
        print("Centroid:", prop.centroid)
        print("Alan:", prop.area)
        print("Yoğunluk Ortalaması:", prop.mean_intensity)
        # Check if minor_axis_length is zero before division
        if prop.minor_axis_length != 0:
            print("Eksen Oranı:", prop.major_axis_length / prop.minor_axis_length)
        else:
            print("Eksen Oranı: Belirsiz (minor_axis_length sıfır)") # Handle zero minor axis length gracefully.
        print("Dışbükey Alan:", prop.convex_area)
        print("Eylemsizlik Momenti:", prop.inertia_tensor_eigvals)
        print("Çevre Uzunluğu:", prop.perimeter)
        print("Doldurma Oranı:", prop.solidity)
        print("-" * 20)
    #************ÖZ NİTELİKLERE GÖRE FİLTRELEME*********************************************************************

    regions = {}
    for prop in props:
            regions[prop.label] = {
                "centroid": prop.centroid,
                "area": prop.area,
                "mean_intensity": prop.mean_intensity,
                "minor_axis_length": prop.minor_axis_length,
                "convex_area": prop.convex_area,
                "inertia_tensor_eigvals": prop.inertia_tensor_eigvals,
                "perimeter": prop.perimeter,
                "solidity": prop.solidity
            }

    # 'labels' array'den görüntü boyutları alacağız...
    image_height, image_width = labels.shape

    #**********POST PROCESSİNG 4******************************************************************************************
    #o	Belirlediğniz özniteliklere göre filtreleme yapın. (örn: maks alana sahip ilk iki bölge, if maks alana sahip iki bölge and bölgelerin centroidleri ortada; filter; else try 3. Bölge; return)
    #Belirleyici öznitelik bulunamadıysa. Her görüntüde otomatik olarak label belirleyen algoritmayı yazın

    #filtered_labels = filter_regions(regions, 150, 150) # Pass image width and height to the function
    filtered_labels = filter_regions(regions, image_width, image_height) # Pass image width and height to the function
    filtered_image = np.zeros_like(labels, dtype=np.uint8)

    for label in filtered_labels:
            filtered_image[labels == label] = 255

    plt.figure(figsize=(8, 6))                 # 8 inç x 6 inç boyutlarında çizim için yeni bir şekil oluşturur...
    plt.imshow(filtered_image,cmap='gray')           #Ham görüntüyü (orijinal, işlenmemiş görüntüyü) görüntüleyeceğiz..cmap='gray',renk haritasını tıbbi görüntüler için uygun olan gri tonlamaya ayarlar
    plt.title('Filtre Uygulanmış Görüntü')     #Grafiğin başlığını ayarlar ve ardından görüntü numarasını ekler...
    plt.colorbar()

    #*********OTOMATİK LABEL BELİRLEME***************************************************************************************
    result_img = auto_label_image(closing)

    plt.figure(figsize=(8, 6))                 # 8 inç x 6 inç boyutlarında çizim için yeni bir şekil oluşturur...
    plt.imshow(result_img, cmap='gray')           #Ham görüntüyü (orijinal, işlenmemiş görüntüyü) görüntüleyeceğiz..cmap='gray',renk haritasını tıbbi görüntüler için uygun olan gri tonlamaya ayarlar
    plt.title('3 kanal Uygulanmış Görüntü')     #Grafiğin başlığını ayarlar ve ardından görüntü numarasını ekler...
    plt.colorbar()

    #********MORFOLOJİK İŞLEMLER 5*******************************************************************************************
    #Seçtiğiniz labela sahip filtrelenmiş görüntülere uygun morfolojik işlemleri (structural elemant tipi ve boyutu belirleyerek) uygulayın. Giriş ve çıkış görüntülerini görselleştirin

            # Yapısal elemanı tanımlama (örneğin, 5x5'lik bir dikdörtgen)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))

        # Erosion uygulama
    eroded_image = cv2.erode(result_img, kernel, iterations=1)

        # Dilation uygulama
    dilated_image = cv2.dilate(eroded_image, kernel, iterations=1)

        # Opening uygulama
    opening_image = cv2.morphologyEx(dilated_image, cv2.MORPH_OPEN, kernel)

        # Closing uygulama
    closing_image = cv2.morphologyEx(opening_image, cv2.MORPH_CLOSE, kernel)

        # Giriş ve çıkış görüntülerini çizdireceğiz...
        #plt.figure(figsize=(10, 8))

    plt.figure(figsize=(8, 6))
    plt.imshow(result_img, cmap="gray")
    plt.title("Filtrelenmiş Görüntü")

    plt.figure(figsize=(8, 6))
    plt.imshow(eroded_image, cmap="gray")
    plt.title("Erosion Uygulama")

    plt.figure(figsize=(8, 6))
    plt.imshow(dilated_image, cmap="gray")
    plt.title("Dilation Uygulama")

    plt.figure(figsize=(8, 6))
    plt.imshow(opening_image, cmap="gray")
    plt.title("Opening Uygulama")

    plt.figure(figsize=(8, 6))
    plt.imshow(closing_image, cmap="gray")
    plt.title("Closing Uygulama")

    plt.tight_layout()
    plt.show()


        #result_img = result_img.astype(np.uint8)
    result_img = closing_image.astype(np.uint8)
    transformed_normalized = transformed_normalized.astype(np.uint8)

    #*********** GÖRÜNTÜNÜN ASLI İLE ÇARPIM YAPACAĞIZ...........
    carpim_goruntu1 = cv2.multiply(result_img, equalized_image, scale=1/255.0)
    #carpim_goruntu2 = cv2.multiply(result_img, transformed_normalized, scale=1/255.0)

    # Sonucu görüntülüyoruz..
    plt.figure(figsize=(16, 12))                 # 16 inç x 12 inç boyutlarında çizim için yeni bir şekil oluşturur...
    plt.imshow(carpim_goruntu1, cmap='gray')
    plt.title("ASIL GÖRÜNTÜ ile ÇARPILMIŞ GÖRÜNTÜ")
    plt.colorbar()

    plt.show()
    #**********************************************
    # Sonucu görüntülüyoruz...
    #plt.figure(figsize=(16, 12))                 # 16 inç x 12 inç boyutlarında çizim için yeni bir şekil oluşturur...
    #plt.imshow(carpim_goruntu2, cmap='gray')
    #plt.title("Transform görüntü ile çarpılmış görüntü")
    #plt.colorbar()

    #plt.show()

